In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv(r'E:\BDT\5001\kaggle comp\train.csv',sep=',',header=0)
#df0 = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

df_t = pd.read_csv(r'E:\BDT\5001\kaggle comp\test.csv',sep=',',header=0)

## Bag of Words

In [3]:
df['new'] = df['genres']+df['categories']+df['tags']
df_t['new'] = df_t['genres']+df_t['categories']+df_t['tags']

In [4]:
text1 = list(df['new'])
text2 = list(df_t['new'])
text = text1 + text2


In [5]:
def BOW(text):
    max_features=100
    # 切割词袋 删除英文停用词
    #vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=max_features,stop_words='english',lowercase=True)
    vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=max_features, stop_words='english', lowercase=True)
    #vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=5000, stop_words=None, lowercase=True)

    print( "vectorizer 参数:")
    print (vectorizer)
    # 该类会统计每个词语的tf-idf权值
    transformer = TfidfTransformer()
    # 使用2-gram和TFIDF处理
    tags = transformer.fit_transform(vectorizer.fit_transform(text))
    #tags = vectorizer.fit_transform(text)


    return tags

In [6]:
tags = BOW(text)
tags_features = tags.toarray()
tags_features.shape

vectorizer 参数:
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


(447, 100)

In [7]:
tags_features_train = tags_features[0:357]
tags_features_test = tags_features[357:447]
print(tags_features_train.shape)
print(tags_features_test.shape)

(357, 100)
(90, 100)


In [8]:
df1 = pd.DataFrame(tags_features_train)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.0,0.000000,0.000000,0.223134,0.0,0.143086,0.0,0.0,0.0,...,0.0,0.0,0.123381,0.0,0.0,0.0,0.311098,0.0,0.0,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0.177529,0.0,0.000000,0.000000,0.183692,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.101572,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0.000000,0.0,0.110117,0.252122,0.108101,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.119548,0.0,0.0,0.0,0.000000,0.0,0.0,0.169237
4,0.000000,0.0,0.088243,0.202039,0.086627,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.095800,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [9]:
df.head()

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,new
0,0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018/7/2,2013/12/10,372,96,"Adventure,Casual,IndieSingle-player,Steam Trad..."
1,1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016/11/26,2015/8/12,23,0,"RPGSingle-player,Partial Controller SupportMod..."
2,2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018/7/2,2014/1/28,3018,663,"Adventure,Casual,IndieSingle-player,Full contr..."
3,3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016/11/28,2010/3/31,63078,1746,"Action,RPGSingle-player,Multi-player,Steam Ach..."
4,4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018/3/4,2012/7/30,8841,523,"Action,Indie,StrategySingle-player,Co-op,Steam..."


## 处理时间信息

In [26]:
df['purchase_date'] =pd.to_datetime(df['purchase_date'])
df['release_date'] =pd.to_datetime(df['release_date'])

In [27]:
buy_time_fromRelease = (df['purchase_date'] - df['release_date']).dt.days

new_old = [ 0 if days > 365 else 1 for days in buy_time_fromRelease]

new_old_games= pd.DataFrame(new_old, columns=['new_old_games']) 
new_old_games.head()
#new_old_games.count()

,new_old_games
0,0
1,0
2,0
3,0
4,0


In [28]:
data = df[['playtime_forever','price']]

data = pd.concat([data, new_old_games] ,sort=False, axis=1 )

data.head()


,playtime_forever,price,new_old_games
0,0.000000,3700,0
1,0.016667,0,0
2,0.000000,5000,0
3,1.533333,9900,0
4,22.333333,4800,0


## Tags and positive review rate

In [15]:
data1 = df.drop(columns=['playtime_forever','price', 'is_free','genres', 'categories','tags' , 'purchase_date','release_date'])
data1.head()

,id,total_positive_reviews,total_negative_reviews,new
0,0,372,96,"Adventure,Casual,IndieSingle-player,Steam Trad..."
1,1,23,0,"RPGSingle-player,Partial Controller SupportMod..."
2,2,3018,663,"Adventure,Casual,IndieSingle-player,Full contr..."
3,3,63078,1746,"Action,RPGSingle-player,Multi-player,Steam Ach..."
4,4,8841,523,"Action,Indie,StrategySingle-player,Co-op,Steam..."


In [16]:
data1['pos_rev_rate'] = data1['total_positive_reviews']/(data1['total_positive_reviews']+data1['total_negative_reviews']) 

In [18]:
data2 = data1.drop(columns=['total_positive_reviews','total_negative_reviews'])
data2.head()

,id,new,pos_rev_rate
0,0,"Adventure,Casual,IndieSingle-player,Steam Trad...",0.794872
1,1,"RPGSingle-player,Partial Controller SupportMod...",1.000000
2,2,"Adventure,Casual,IndieSingle-player,Full contr...",0.819886
3,3,"Action,RPGSingle-player,Multi-player,Steam Ach...",0.973066
4,4,"Action,Indie,StrategySingle-player,Co-op,Steam...",0.944148


In [19]:
data3 = pd.concat([data2, df1] ,sort=False,axis = 1)
#data3 = data3.fillna(0)

data3 = data3.drop(columns=['new'])
data3.head()

,id,pos_rev_rate,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0,0.794872,0.000000,0.0,0.000000,0.000000,0.223134,0.0,0.143086,0.0,...,0.0,0.0,0.123381,0.0,0.0,0.0,0.311098,0.0,0.0,0.000000
1,1,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,2,0.819886,0.177529,0.0,0.000000,0.000000,0.183692,0.0,0.000000,0.0,...,0.0,0.0,0.101572,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,3,0.973066,0.000000,0.0,0.110117,0.252122,0.108101,0.0,0.000000,0.0,...,0.0,0.0,0.119548,0.0,0.0,0.0,0.000000,0.0,0.0,0.169237
4,4,0.944148,0.000000,0.0,0.088243,0.202039,0.086627,0.0,0.000000,0.0,...,0.0,0.0,0.095800,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


## 整合所有features

In [20]:
data_preprocessed = pd.concat([data3, data] ,sort=False, axis=1 )
data_preprocessed.head()

,id,pos_rev_rate,0,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,playtime_forever,price,new_old_games
0,0,0.794872,0.000000,0.0,0.000000,0.000000,0.223134,0.0,0.143086,0.0,...,0.0,0.0,0.0,0.311098,0.0,0.0,0.000000,0.000000,3700,0
1,1,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.016667,0,0
2,2,0.819886,0.177529,0.0,0.000000,0.000000,0.183692,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,5000,0
3,3,0.973066,0.000000,0.0,0.110117,0.252122,0.108101,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.169237,1.533333,9900,0
4,4,0.944148,0.000000,0.0,0.088243,0.202039,0.086627,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,22.333333,4800,0


In [21]:
data_preprocessed['price'].max()
data_preprocessed['price'].mean()

71232.77030812325

In [22]:
data_preprocessed = data_preprocessed.drop(columns=['id'])
data_preprocessed['price'] = (data_preprocessed['price']-data_preprocessed['price'].mean())/data_preprocessed['price'].max()
data_preprocessed.head()

,pos_rev_rate,0,1,2,3,4,5,6,7,8,...,93,94,95,96,97,98,99,playtime_forever,price,new_old_games
0,0.794872,0.000000,0.0,0.000000,0.000000,0.223134,0.0,0.143086,0.0,0.0,...,0.0,0.0,0.0,0.311098,0.0,0.0,0.000000,0.000000,-0.004221,0
1,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.016667,-0.004452,0
2,0.819886,0.177529,0.0,0.000000,0.000000,0.183692,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,-0.004140,0
3,0.973066,0.000000,0.0,0.110117,0.252122,0.108101,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.169237,1.533333,-0.003833,0
4,0.944148,0.000000,0.0,0.088243,0.202039,0.086627,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,22.333333,-0.004152,0


In [23]:
data_preprocessed.to_csv("kaggle_data.csv",index=False,sep=',')

## Preprocess text data in same way

In [30]:
#test_preprocess

df_t['purchase_date'] =pd.to_datetime(df_t['purchase_date'])
df_t['release_date'] =pd.to_datetime(df_t['release_date'])
    
buy_time_fromRelease = (df_t['purchase_date'] - df_t['release_date']).dt.days

new_old = [ 0 if days > 365 else 1 for days in buy_time_fromRelease]

new_old_games= pd.DataFrame(new_old, columns=['new_old_games']) 
    
data = df_t[['price']]
data = pd.concat([data, new_old_games] ,sort=False, axis=1 )
data.count()    

price            90
new_old_games    90
dtype: int64

In [31]:
data1 = df_t.drop(columns=['price', 'is_free','genres', 'categories','tags' , 'purchase_date','release_date'])
data1['pos_rev_rate'] = data1['total_positive_reviews']/(data1['total_positive_reviews']+data1['total_negative_reviews']) 
data2 = data1.drop(columns=['total_positive_reviews','total_negative_reviews'])
    
#get tags_features_test
df1 = pd.DataFrame(tags_features_test)
data3 = pd.concat([data2, df1] ,sort=False,axis = 1)
data3 = data3.drop(columns=['new'])
    
data_preprocessed = pd.concat([data3, data] ,sort=False, axis=1 )
data_preprocessed = data_preprocessed.drop(columns=['id'])
data_preprocessed['price'] = (data_preprocessed['price']-data_preprocessed['price'].mean())/data_preprocessed['price'].max()
data_preprocessed.to_csv("test_preprocessed.csv",index=False,sep=',')
    